In [2]:
!pip install medmnist
!pip install medmnist --upgrade
!pip install openai

In [ ]:
import os
import torch
from torch import optim
import torchvision
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import pandas as pd
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from skimage.util import montage
import torch.optim as optim
from torchvision import models, transforms

In [3]:
# Function to load dataset
def load_dataset(button):
    global dataset, images, labels
    
    path = '/home/mpcr/Desktop/datasets/pneumoniamnist/train_images.npy'
    data = np.load(path)

    # Preview a single image from the dataset
    image = data[0]  # Get the first image
    plt.imshow(image, cmap='gray')
    plt.title("Single Image")
    plt.axis('off')
    plt.show()

    # Create a montage of images from the dataset with labels
    num_images = data.shape[0]  # Number of images in the dataset
    images = data
    labels = np.arange(num_images)

    fig, axes = plt.subplots(5, 5, figsize=(8, 8))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(images[i], cmap='gray')
        ax.set_title(f"Label: {labels[i]}")
        ax.axis('off')
    plt.suptitle("Montage of Images with Labels")
    plt.tight_layout()
    plt.show()

    dataset = list(zip(images, labels))  # Create dataset as a list of tuples

In [4]:
# Function to generate histogram
# Declare a variable to keep track of whether the histogram has been generated or not
histogram_generated = False

def generate_histogram(button):
    global histogram_generated

    if not histogram_generated:
        if labels is not None:
            # Convert labels to a NumPy array
            labels_array = np.array(labels)

            # Generate histogram
            plt.figure(figsize=(10, 6))
            plt.hist(labels_array, bins=10, edgecolor='black')
            plt.xlabel('Label')
            plt.ylabel('Frequency')
            plt.title('Histogram of MNIST Dataset')
            plt.show()

            histogram_generated = True
        else:
            print("Please load the dataset first.")
    else:
        print("Histogram has already been generated. Click 'Generate Histogram' only once.")

In [5]:
# Function to view RGB channels
def view_rgb_channels(button):
    global dataset

    if dataset is not None:
        # Get a random image from the dataset
        image, _ = dataset[np.random.randint(len(dataset))]

        # Check if the image is grayscale or RGB
        if len(image.shape) == 2:
            # Grayscale image, duplicate the channel for RGB visualization
            image = np.stack([image]*3, axis=-1)

        # Split the image into RGB channels
        red_channel = image[:, :, 0]
        green_channel = image[:, :, 1]
        blue_channel = image[:, :, 2]

        # Plot the RGB channels separately
        fig, axes = plt.subplots(1, 4, figsize=(16, 4))

        # Display the original image
        axes[0].imshow(image, cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        # Display the RGB channels
        axes[1].imshow(red_channel, cmap='gray')
        axes[1].set_title('Red Channel')
        axes[1].axis('off')
        axes[2].imshow(green_channel, cmap='gray')
        axes[2].set_title('Green Channel')
        axes[2].axis('off')
        axes[3].imshow(blue_channel, cmap='gray')
        axes[3].set_title('Blue Channel')
        axes[3].axis('off')

        plt.tight_layout()
        plt.show()
    else:
        print("Please load the dataset first.")

In [6]:
# Function to train model

# Global variables
histogram_generated = False
dataset = None
images = None
labels = None
model = None

def train_neural_network(button):
    global model
    # Define data directory and check if the directories exist
    data_dir = '/home/mpcr/Desktop/datasets/pneumoniamnist'
    sub_dirs = ['train', 'val', 'test']

    for dir in sub_dirs:
        os.makedirs(os.path.join(data_dir, dir, 'pneumonia'), exist_ok=True)
        os.makedirs(os.path.join(data_dir, dir, 'normal'), exist_ok=True)

    # Load your .npy file
    images = np.load('/home/mpcr/Desktop/datasets/pneumoniamnist/train_images.npy')

    # Define your class labels here based on your specific dataset
    labels = np.load('/home/mpcr/Desktop/datasets/pneumoniamnist/train_labels.npy')

    # Split data into train, validation, and test sets
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.30, random_state=42)
    val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.50, random_state=42)

    # Create directories for each set and class
    for name in ['train', 'val', 'test']:
        os.makedirs(f'/home/mpcr/Desktop/datasets/pneumoniamnist/{name}/pneumonia', exist_ok=True)
        os.makedirs(f'/home/mpcr/Desktop/datasets/pneumoniamnist/{name}/normal', exist_ok=True)

    # Save images to the appropriate directory
    image_data = {'train': (train_images, train_labels), 'val': (val_images, val_labels), 'test': (test_images, test_labels)}

    for name, (images, labels) in image_data.items():
        for i, image in enumerate(images):
            image = Image.fromarray(image)
            if labels[i] == 0:
                image.save(f'/home/mpcr/Desktop/datasets/pneumoniamnist/{name}/normal/normal_{i}.png')
            else:
                image.save(f'/home/mpcr/Desktop/datasets/pneumoniamnist/{name}/pneumonia/pneumonia_{i}.png')

    # Define the transforms
    transforms_ = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]),
        'test': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]),
    }

    # Load the dataset from the folders
    image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), transforms_[x]) for x in ['train', 'val', 'test']}

    # Create DataLoaders
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

    # Load the pre-trained model
    model = models.resnet50(pretrained=True)

    # Freeze the parameters of the model
    for param in model.parameters():
        param.requires_grad = False

    # Change the final layer of the model
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 500),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(500, 2),
        nn.LogSoftmax(dim=1)
    )

    # Define the loss function and the optimizer
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.fc.parameters())

    # Training loop
    for epoch in range(1):  # Loop over the dataset multiple times
        running_loss = 0.0
        for images, labels in dataloaders['train']:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print loss
            running_loss += loss.item()

        # Print epoch loss
        print(f'Epoch [{epoch+1}/2], Loss: {running_loss/len(dataloaders["train"])}')

    print('Finished Training')

In [7]:
# Function to save the model
def save_model(button):
    global model  # Reference the global variable

    # Check if the model is trained and exists
    if model is None:
        print("No model exists. Please train a model first.")
        return

    # Define the model name
    model_name = 'model.pt'

    # Save the model state_dict
    torch.save(model.state_dict(), model_name)

    print(f'Model saved as {model_name}')


# Function to load model
def load_model(button):
    global model  # Reference the global variable

    # Specify the name of your saved model
    model_name = 'model.pt'

    # Ensure your model architecture is defined
    model = models.resnet50(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 500),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(500, 2),
        nn.LogSoftmax(dim=1)
    )

    # Load the state dict previously saved
    model.load_state_dict(torch.load(model_name))

    print(f'Model loaded from {model_name}')


In [8]:
# Function to test neural network
# Global variables
histogram_generated = False
dataset = None
images = None
labels = None
model = None

# Define the transform
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def test_neural_network(button):
    global model  # Reference the global variable

    # Define the directory paths
    test_directory = '/home/mpcr/Desktop/datasets/pneumoniamnist/test'

    # Load the test dataset from the folder
    test_dataset = ImageFolder(test_directory, transform=test_transform)

    # Create a DataLoader for the test dataset
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))


In [9]:
# Function for Result
def result_qa(button):
    global model  # Reference the global variable

    # Load the model
    model_name = 'model.pt'
    model = models.resnet50(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 500),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(500, 2),
        nn.LogSoftmax(dim=1)
    )
    model.load_state_dict(torch.load(model_name))
    model.eval()

    # Define the path to the image you want to test
    image_path = "/home/mpcr/Desktop/datasets/pneumoniamnist/test/pneumonia/pneumonia_10.png"

    # Load the image
    image = Image.open(image_path)

    # Convert grayscale image to RGB
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Apply the transform to the image
    image = test_transform(image).unsqueeze(0)

    # Predict the class of the image
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)

    if predicted.item() == 0:
        print("The model predicts that this image is of a normal lung.")
    else:
        print("The model predicts that this image is of a pneumonia-affected lung.")

In [10]:
# Function for Q&A
def q_and_a(event):
    # Code for Q&A functionality goes here
    print("Performing Q&A...")




load_dataset_button = widgets.Button(description="Load Dataset")
generate_histogram_button = widgets.Button(description="Generate Histogram")
view_rgb_channels_button = widgets.Button(description="View RGB Channels")
train_neural_network_button = widgets.Button(description="Train Neural Network")
save_model_button = widgets.Button(description="Save Model")
load_model_button = widgets.Button(description="Load Model")
test_neural_network_button = widgets.Button(description="Test Accuracy")
result_button = widgets.Button(description="Result")
q_and_a_button = widgets.Button(description="Q&A")

# Set button click event handlers
load_dataset_button.on_click(load_dataset)
generate_histogram_button.on_click(generate_histogram)
view_rgb_channels_button.on_click(view_rgb_channels)
train_neural_network_button.on_click(train_neural_network)
save_model_button.on_click(save_model)
load_model_button.on_click(load_model)
test_neural_network_button.on_click(test_neural_network)
result_button.on_click(result_qa)
q_and_a_button.on_click(q_and_a)


# Create tabs for each functionality
tab_contents = [
    widgets.VBox([load_dataset_button]),
    widgets.VBox([generate_histogram_button]),
    widgets.VBox([view_rgb_channels_button]),
    widgets.VBox([train_neural_network_button]),
    widgets.VBox([save_model_button]),
    widgets.VBox([load_model_button]),
    widgets.VBox([test_neural_network_button]),
    widgets.VBox([result_button]),
    widgets.VBox([q_and_a_button])
]
tab_titles = ['Load Dataset', 'Generate Histogram', 'View RGB Channels', 'Train Neural Network', 'Save Model',
              'Load Model', 'Test Neural Network', 'Result', 'Q&A']

# Create the tabs
tabs = widgets.Tab()
tabs.children = tab_contents

# Set tab titles
for i in range(len(tabs.children)):
    tabs.set_title(i, tab_titles[i])

# Display the tabs
display(tabs)